In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [5]:
# Helper funtion to show an image with cv2
def cv_show_img(title, image, wait=0):
    cv2.namedWindow(title)
    cv2.startWindowThread()
    cv2.imshow(title, image)
    cv2.waitKey(wait)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# Helper funtion to show multiple images at the same time
def cv_show_mult_img(titleArr, imageArr, wait=0):
    for i in range(len(titleArr)):
        cv2.namedWindow(titleArr[i])
        cv2.startWindowThread()
        cv2.imshow(titleArr[i], imageArr[i])
    cv2.waitKey(wait)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [6]:
img = cv2.imread('./images/input.jpg')

## Shifting Images

In [7]:
# Lets shift by a quarter of the image
height, width = img.shape[:2]
quarter_height, quarter_width = height/4, width/4

# Define translation matrix
#     | 1 0 Tx |
# T = |        |
#     | 0 1 Ty |
T = np.float32([[1, 0, quarter_width], [0, 1, quarter_height]])

# warpAffine to make an affine transformation using the matrix T
img_translated = cv2.warpAffine(img, T, (width, height))
cv_show_mult_img(['Original','Translated'],[img,img_translated])

In [9]:
# Lets see T
print(T)

[[  1.     0.   242.  ]
 [  0.     1.   149.25]]


## Rotating Images

In [10]:
# rotate image about it's center
# Get the rotation matrix
rot_matrix = cv2.getRotationMatrix2D(
    (width/2,height/2),     # tuple of the center point of the image coordinates
    90,     # degrees to rotate
    0.5     # scale of the resulting image
)

rot_image = cv2.warpAffine(img, rot_matrix, (width, height))

cv_show_mult_img(['Original','Rotated'],[img,rot_image])

## Removing the Resulting Black Space

In [11]:
# All we have to do is transpose the image for 90 degree rotation, flip it for 180 degree
rot_img_90 = cv2.transpose(img)
rot_img_180_horz = cv2.flip(img, 1)
rot_img_180_vert = cv2.flip(img, -1)
cv_show_mult_img(['Original','90 Degrees','180 - Horizontal','180 - Vertical'],[img, rot_img_90, rot_img_180_horz, rot_img_180_vert])

## Scaling, Re-Sizing, Interpolation, Cropping

Interpolation is the super-set of all of these actions - it refers to trying to basically guess what a changed image should look like. 
When Interpolating with cv2, we need to specify a an interpolation method. Here are a few examples:
- cv2.INTER_AREA - good for shrinking or down sampling
- cv2.INTER_NEAREST - fastest
- cv2.INTER_LINEAR - good for zooming or up-sampling (also default)
- cv2.INTER_CUBIC - Better
- cv2.INTER_LANCZOS4 - Best

In [13]:
oxf = cv2.imread('./images/oxford.jpg')

In [15]:
# Re-Scale to 75% of the original size
oxf_scaled = cv2.resize(
    oxf,
    None,       # Argument for resulting image size - can leave none when you're rescaling based on pcts
    fx=0.75,    # x scale factor
    fy=0.75     # y scale factor
    # interpolation=cv2.INTER_LINEAR
)
cv_show_mult_img(['Original','75% Scaled'],[oxf,oxf_scaled])

In [16]:
# How about doubling with varioius different interpolation methods
oxf_cubic = cv2.resize(
    oxf,
    None,
    fx=2,
    fy=2,
    interpolation=cv2.INTER_CUBIC
)

oxf_nearest = cv2.resize(
    oxf,
    None,
    fx=2,
    fy=2,
    interpolation=cv2.INTER_NEAREST
)

cv_show_mult_img(['Original','Cubic Interpolation','Nearest Interpolation'],[oxf,oxf_cubic,oxf_nearest])

In [17]:
# Re-size to specific dimensions
oxf_area = cv2.resize(
    oxf,
    (900,400),
    interpolation=cv2.INTER_AREA
)

oxf_prop = cv2.resize(
    oxf,
    None,
    fx=2,
    fy=0.5,
    interpolation=cv2.INTER_AREA
)

cv_show_mult_img(['Original','Skewed - Exact','Skewed - Proportion'],[oxf,oxf_area,oxf_prop])

## Image Pyramiding

Information scaling - good for object detection

In [18]:
oxf_small = cv2.pyrDown(oxf)
oxf_large = cv2.pyrUp(oxf_small)

# You should see information loss in the re-sized-up image

cv_show_mult_img(['Original','Small','large'],[oxf,oxf_small,oxf_large])

## Cropping

In [20]:
# Let's crop the middle 50% of the image
height, width = oxf.shape[:2]

# get the top left point of the cropping window we want 
# 25% down and right from the top left corner or 75% up and left from the bottom right corner
# And also get the bottom right of the cropping window
start_row, start_col = int(height * 0.25), int(width * 0.25)
end_row, end_col = int(height * 0.75), int(width * 0.75)

# now get the cropped rectangle - we just slice the area we want
oxf_cropped = oxf[start_row:end_row, start_col:end_col]

# Also display the area we are cropping
oxf_rect = oxf.copy()
cv2.rectangle(oxf_rect,(start_col,start_row),[end_col,end_row],(0,255,255),10)

#display images
cv_show_mult_img(['Original w/ Cropping','Cropped'],[oxf_rect,oxf_cropped])